# Lógica do programa

## Passo a passo

- Criar navegador<br>
<br>
- Importar e visualizar a base de dados<br>
<br>
- Para cada item da base de dados:
    - Procurar produto no google shopping
        - Verificar se algum produto entá dentro da faixa de preço
        
    - Procurar produto no google shopping
        - Verificar se algum produto entá dentro da faixa de preço<br>
<br>
- Salvar ofertas boas em um DataFrame<br>
<br>
- Exportar para o excel<br>
<br>
- Enviar por e-mail o resultado da tabela
        

### Passo 1 - Criando Navegador:

In [155]:
# Importando módulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
import yagmail

# Criando navegador
nav = webdriver.Chrome()

### Passo 2 - Importar e Visualizar a base de dados:

In [156]:
# Criando base de dadpos
tabela_produtos = pd.read_excel("buscas.xlsx")  # Necessário estar na mesma pasta
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


### Passo 3 - Automatizando busca na Web

#### Criando a função de pesquisa do Google  Shopping

In [157]:
def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):
    # Entrar no goole
    nav.get("https://www.google.com.br/")

    # Tratando possíveis erros da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_max = float(preco_max)
    preco_min = float(preco_min)
    
    # Pesquisar nome do produto
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # Entrar no google shopping
    elementos = nav.find_elements(By.CLASS_NAME, "hdtb-mitem")  
    for item in elementos:                                      
        if "Shopping" in item.text:                             
            item.click()
            break

    # Pegar a lista de "resultados" da busca 
    lista_resultados = nav.find_elements(By.CLASS_NAME, "sh-dgr__grid-result")
    
    # Para cada "resultado", ele vai verificar se o "resultado" corresponde a todas as condições
    lista_ofertas = [] # Lista reposta da função

    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, "Xjkr3b").text
        nome = nome.lower()

        # Verificação do nome - Se possue algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        # Verificação do nome - Se possue todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        # Se tem_termos_banidos = False e tem_todos_termos_produto = True
        if not tem_termos_banidos and tem_todos_termos_produto:
            try: # Caso o preço não seja apenas numérico
                preco = resultado.find_element(By.CLASS_NAME, "a8Pemb").text
                # Tratando erros no preço e transformando para "float"
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                
                # Verificando se o preço está dentro do mínimo e máximo
                if preco_min <= preco <= preco_max: 
                    elemento_link = resultado.find_element(By.CLASS_NAME, "aULzUe") # Pegando elemento filho
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')       # Pegando elemento pai
                    link = elemento_pai.get_attribute("href")                       # Pegando link
                    lista_ofertas.append((nome, preco, link))
            except: # Pula esse produto
                continue

    return lista_ofertas


#### Criando a função de pesquisa do Buscapé

In [158]:
def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):
    # Tratar possíveis erros da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_max = float(preco_max)
    preco_min = float(preco_min)
    
    # Entrar no Buscapé
    nav.get("https://www.buscape.com.br/")
    
    # Pesquisar pelo produto
    nav.find_element(By.CLASS_NAME, "search-bar__text-box").send_keys(produto, Keys.ENTER)
    
    # Esperar o site carregar
    time.sleep(5)
    
    # Pegar a lista de "resultados" da busca
    lista_resultados = nav.find_elements(By.CLASS_NAME, "Cell_Content__1630r")
    
    # Para cada "resultado":
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, "CellPrice_MainValue__3s0iP").text
            nome = resultado.get_attribute("title")
            nome = nome.lower()
            link = resultado.get_attribute("href")
            
        # Verificação do nome - Se possue algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

        # Verificação do nome - Se possue todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

        # Se tem_termos_banidos = False e tem_todos_termos_produto = True
            if not tem_termos_banidos and tem_todos_termos_produto:
                try: # Caso o preço não seja apenas numérico
                    # Tratando erros no preço e transformando para "float"
                    preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                    preco = float(preco)
                    # Verificar se está dentro da faixa de preço
                    if preco_min <= preco <= preco_max: 
                        lista_ofertas.append((nome, preco, link))
                except: # Pula esse produto
                    continue
        except:
            pass
        
    # Retornar a lista de ofertas
    return lista_ofertas


### Passo 4 - Contrução da lista de ofertas encontradas

In [159]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_min = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_max = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns = ["Produto", "Preço", "Link"])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else: 
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns = ["Produto", "Preço", "Link"])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
        
display(tabela_ofertas)


,Produto,Preço,Link
0,"iphone xr preto, com tela de 6,1, 4g, 64 gb e ...",3099.00,https://www.google.com.br/url?url=https://www....
0,smartphone apple iphone xr 64gb ios 12.0 mp,3039.00,https://www.buscape.com.br/celular/smartphone-...
1,"iphone xs space gray, com tela de 5,8, 4g, 64 ...",3347.56,https://www.buscape.com.br/lead?oid=714484000&...
2,"iphone xs space gray, com tela de 5,8, 4g, 64 ...",3347.56,https://www.buscape.com.br/lead?oid=714417477&...
0,placa de vídeo gigabyte geforce rtx 3060 eagle...,4199.90,https://www.google.com.br/url?url=https://www....
1,placa de vídeo palit geforce rtx 3060 12gb - g...,4470.57,https://www.google.com.br/url?url=https://www....
2,"rtx 3060 ventus 2x 12g oc, 15 gbps, 12gb gddr6...",4299.90,https://www.google.com.br/url?url=https://www....
3,placa de video colorful igame geforce rtx 3060...,4229.10,https://www.google.com.br/url?url=https://www....
4,placa de vídeo gainward geforce rtx 3060 ghost...,4399.00,https://www.google.com.br/url?url=https://prod...
5,placa de vídeo colorful igame geforce rtx 3060...,4349.13,https://www.google.com.br/url?url=https://www....


### Passo 5 - Exportando DataFrame para o Excel

In [160]:
tabela_ofertas = tabela_ofertas.reset_index(drop = True)
tabela_ofertas.to_excel("Ofertas.xlsx", index = False)

### Passo 6 - Enviando E-mail

In [161]:
usuario = yagmail.SMTP(user = "inserir e-mail do remetente", password = "Inserir senha da conta do e-mail")

# Verificando se existe alguma oferta na tabela de ofertas
if len(tabela_ofertas.index) > 0:
    para = "inserir e-mail do destinatário"
    assunto = "Produto(s) encontrado(s) na faixa de preço desejada" 
    corpo_email = f'''
    <p>Prezados</P>
    <p>Encontramos alguns produtos em ofertas dentro da faixa de preço desejada. Segue tabela com detalhes</P>
    {tabela_ofertas.to_html(index = False)}
    <p>Qualquer dúvida estou a disposição</p>
    <p>Att,</p>
    <p>Alan</p>
    '''
    usuario.send(to = para, subject = assunto, contents = corpo_email)

nav.quit()